In [1]:
import arcgis, arcpy
from arcgis.gis import GIS

In [2]:
point_feature = r'Database Connections\FIRS_Steward@maint.gissqlkc.sde\maint.FIRS.d_flowmonitors_point_edit'
related_table = r'Database Connections\FIRS_Steward@maint.gissqlkc.sde\maint.FIRS.t_flowmonitors_relatedrecords_edit'

In [3]:
password = open("C:\\Users\\kimvo\\OneDrive - King County\\Documents\\getpass.txt").readline()
gis = GIS("http://kingcounty.maps.arcgis.com/home", "kimvo_kingcounty", password)

In [4]:
search = gis.content.search('title: METER','Feature Layer')[0]
MIlayer = search.layers[0]
MIdata = MIlayer.query().features

In [5]:
ago_fields = ["METER_ID","ID_FAC","DEPTH_FAC","PROJECT","S_DATE","HEIGHT","WIDTH","PIPE_SHAPE",\
              "MATERIAL","REQUESTOR","METER_TYPE","SENSOR1","SENSOR1_OFFSET","SENSOR2",\
              "SENSOR2_OFFSET","TRIPOD_REQUIRED","SAFETY_TRAFFIC","HYDRAULICS","OTHER_INFO",\
              "SURCHARGE","TIMEOFDATE","DEPTH_OF_FLOW","DEPTH_DEVIA","VELOCITY","SILT","SILT_DEVIA",\
              "E_DATE"]
rt_fields = ["METER_ID","ID_FAC","DEPTH_FAC","PROJECT","S_DATE","HEIGHT","WIDTH","PIPE_SHAPE",\
              "MATERIAL","REQUESTOR","METER_TYPE","SENSOR1","SENSOR1_OFF","SENSOR2",\
              "SENSOR2_OFF","TRIPOD_REQ","SAFE_TRAFF","HYDROLICS","OTHERINFO",\
              "SURCHARGE","TIMEOFDAY","DEPTH_FLOW","DEPTH_DEVIATION","VELOCITY","SILT", "SILT_DEVIATION",\
              "E_DATE"]

In [6]:
# Backup fc and related table before doing editing
loc = r'Q:\WTD\DataDev\FlowMonitors\Shapes\Backups'
date = str(datetime.datetime.now())[0:10].replace("-", "") #20180524
fcName = point_feature.split("\\")[-1:][0].split(".")[-1:][0]
tbName = related_table.split("\\")[-1:][0].split(".")[-1:][0]

fcOutput = loc + "\\" + fcName + "_" + date + ".xml"
arcpy.ExportXMLWorkspaceDocument_management(point_feature, fcOutput, "DATA", "BINARY", "METADATA")
print("Backup saved for feature class:", fcOutput)

tbOutput = loc + "\\" + tbName + "_" + date + ".xml"
arcpy.ExportXMLWorkspaceDocument_management(related_table, tbOutput, "DATA", "BINARY", "METADATA")
print("Backup saved for related table:", tbOutput)

Backup saved for feature class: Q:\WTD\DataDev\FlowMonitors\Shapes\Backups\d_flowmonitors_point_edit_20190708.xml
Backup saved for related table: Q:\WTD\DataDev\FlowMonitors\Shapes\Backups\t_flowmonitors_relatedrecords_edit_20190708.xml


In [7]:
import csv, datetime
ts = datetime.datetime.now().strftime("%Y%m%d")
log_loc = r'\\dnrp1\projects\WTD\DataDev\FlowMonitors\Documents\MI_PostProcess_Logs'
# set fields for insert cursor for feature class
fc_fields = ['METER_ID', 'SHAPE@XY']

In [8]:
from pyproj import Proj, transform

def convertXY(x1, y1):
    inProj = Proj(init='EPSG:3857')
    outProj = Proj(init='EPSG:2926')#32048 NAD27
    x2,y2 = transform(inProj,outProj,x1,y1)
    return (x2,y2)

In [9]:
arcpy.env.workspace = "Database Connections\\FIRS_Steward@maint.gissqlkc.sde"
edit = arcpy.da.Editor(arcpy.env.workspace)
edit.startEditing(False, True)
edit.startOperation()

In [10]:
# New point with new record
f_cursor = arcpy.da.InsertCursor(point_feature, fc_fields)
all_existing_meterID = [row[0] for row in arcpy.da.SearchCursor(point_feature, ['METER_ID'])]
with open(log_loc + '\\Log_' + ts + '.csv', 'a', newline ='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header for csv log
    filewriter.writerow(['New Meter with New Record'])
    filewriter.writerow(rt_fields)
    # look for new ACTIVE meter with new record
    for i in MIdata:
        if i.attributes['METER_ID'] not in all_existing_meterID and i.attributes['S_DATE'] is not None:
            # insert shape and meter ID to fc
            x = i.geometry['x']
            y = i.geometry['y']
            f_cursor.insertRow([i.attributes['METER_ID'], convertXY(x,y)])
            del f_cursor
            print('New active METER_ID:', i.attributes['METER_ID'])
            # insert attributes to related table
            value = []
            t_cursor = arcpy.da.InsertCursor(related_table, rt_fields)
            for a in ago_fields:
                if type(i.attributes[a]) == str and "," in i.attributes[a]:
                    i.attributes[a] = i.attributes[a].replace(",",' ')
                if a == "TIMEOFDATE" or a == 'S_DATE':
                    date_str = time.strftime('%m/%d/%Y %H:%M:%S', time.localtime((i.attributes[a])/1000.0))
                    value.append(date_str)
                else:
                    value.append(i.attributes[a])
            t_cursor.insertRow(value)
            # write record to csv log
            filewriter.writerow(value)
            del t_cursor
            print('Added attributes to related table')

In [11]:
# Existing location, new attributes/recors
act_exiting_meterID = [row[0] for row in arcpy.da.SearchCursor(related_table, ['METER_ID'], \
                                                               where_clause = "E_DATE IS NULL")]
t_cursor = arcpy.da.InsertCursor(related_table, rt_fields)
with open(log_loc + '\\Log_' + ts + '.csv', 'a', newline ='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header for csv log
    filewriter.writerow(['Historical Meter with New Record'])
    filewriter.writerow(rt_fields)
    # look for new meter with new record but have existing location in fc
    for i in MIdata:
        if i.attributes['METER_ID'] not in act_exiting_meterID \
        and i.attributes['METER_ID'] in all_existing_meterID \
        and i.attributes['E_DATE'] is None \
        and i.attributes['S_DATE'] is not None:
            # insert attributes to related table
            value = []
            for a in ago_fields:
                if type(i.attributes[a]) == str and "," in i.attributes[a]:
                    i.attributes[a] = i.attributes[a].replace(",",' ')
                if (a == "TIMEOFDATE" or a == 'S_DATE') and i.attributes[a] is not None:
                    date_str = time.strftime('%m/%d/%Y %H:%M:%S', time.localtime((i.attributes[a])/1000.0))
                    value.append(date_str)
                else:
                    value.append(i.attributes[a])
            t_cursor.insertRow(value)
            # write record to csv log
            filewriter.writerow(value)
            print('Added new attributes to related table for existing location:',i.attributes['METER_ID'])
del t_cursor

In [12]:
# current active meter got removed
act_exiting_meterID = [row[0] for row in arcpy.da.SearchCursor(related_table, ['METER_ID'], \
                                                               where_clause = "E_DATE IS NULL")]
with open(log_loc + '\\Log_' + ts + '.csv', 'a', newline ='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    # write header for csv log
    filewriter.writerow(['Current Active Meter got removed'])
    filewriter.writerow(rt_fields)
    # look for current active meter just got removed
    for i in MIdata:
        if i.attributes['METER_ID'] in act_exiting_meterID \
        and i.attributes['E_DATE'] is not None \
        and i.attributes['S_DATE'] is not None:
            # insert attributes to related table
            value = []
            t_cursor = arcpy.da.InsertCursor(related_table, rt_fields)
            for a in ago_fields:
                if type(i.attributes[a]) == str and "," in i.attributes[a]:
                    i.attributes[a] = i.attributes[a].replace(",",' ')
                if (a == "TIMEOFDATE" or a == 'S_DATE' or a == 'E_DATE') and i.attributes[a] is not None:
                    date_str = time.strftime('%m/%d/%Y %H:%M:%S', time.localtime((i.attributes[a])/1000.0))
                    value.append(date_str)
                else:
                    value.append(i.attributes[a])
            t_cursor.insertRow(value)
            # write record to csv log
            filewriter.writerow(value)
            del t_cursor
            print('Added E_DATE for removed meter:',i.attributes['METER_ID'] )
            # delete removed meter in MI database
            MIlayer.edit_features(deletes = str(i.attributes['FID']))
            print('Deleted feature from MI:', str(i.attributes['METER_ID']) )

Added E_DATE for removed meter: RNT055
Deleted feature from MI: RNT055


In [13]:
edit.stopOperation()
edit.stopEditing(True)